## DATSCIW261 Assignment 3

**Name: Yi Jin**

**Email: yjin@ischool.berkeley.edu**

**Session: W261-3**

**Week 3**

**Date: Jan 29, 2016**

### HW3.0. ###

**What is a merge sort? Where is it used in Hadoop? **

Mergesort is an efficient sorting algorithm. It divides the unsorted list into n  sublists, each containing 1 element. Then it repeatedly merge sublists to produce new sorted sublists until only one sorted list remaining. Instead of sorting the complete list, sorting partially sorted lists is much more efficient. It generally runs in time O(n log n), given n sorted sublists.

Mergesort is used in hadoop shuffle between map output and reduce input. It merges the output from multiple mappers to a sorted list of key/value pairs for reducer. It is one of the important keys in Hadoop MapReduce.

** How is  a combiner function in the context of Hadoop? ** 

** Give an example where it can be used and justify why it should be used in the context of this problem. **

A combiner is an optional program in MapReduce. It summarize the map output records with the same key. The output(key-value collection) of the combiner will be sent over the network to the actual Reducer task as input. 

The Combiner is used in between the Mapper and the Reducer to reduce the volume of data transfer between Map and Reduce. Usually, the output of the map task is large and the data transferred to the reduce task is high.

There are three ways to implement a combiner: as part of a mapper, a separate combiner program, and as part of a reducer.

WordCount program is an example. The mapper emits a record for each word, so numerous records are created in the map phase. These records need to be sorted and transferred to the reducer. A combiner can be added to aggregate the counts by words to improve the sorting the transfer efficiency.

** What is the Hadoop shuffle? **

The Hadoop shuffle is the heart of MapReduce. It is the process that the system performs the sort, and transfers the map outputs to the reducers as inputs. It includes three phases: Partition, Sort, and Combine.

In [5]:
# start hadoop
!/usr/local/hadoop/sbin/start-all.sh

This script is Deprecated. Instead use start-dfs.sh and start-yarn.sh
16/01/29 23:04:20 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Starting namenodes on [localhost]
localhost: starting namenode, logging to /usr/local/hadoop/logs/hadoop-ubuntu-namenode-ip-172-31-25-140.out
localhost: starting datanode, logging to /usr/local/hadoop/logs/hadoop-ubuntu-datanode-ip-172-31-25-140.out
Starting secondary namenodes [0.0.0.0]
0.0.0.0: starting secondarynamenode, logging to /usr/local/hadoop/logs/hadoop-ubuntu-secondarynamenode-ip-172-31-25-140.out
16/01/29 23:04:35 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
starting yarn daemons
starting resourcemanager, logging to /usr/local/hadoop/logs/yarn-ubuntu-resourcemanager-ip-172-31-25-140.out
localhost: starting nodemanager, logging to /usr/local/hadoop/logs/yarn-ubuntu-nodemanager

### HW3.1 Use Counters to do EDA (exploratory data analysis and to monitor progress) ###

Now, let’s use Hadoop Counters to identify the number of complaints pertaining to debt collection, mortgage and other categories (all other categories get lumped into this one) in the consumer complaints dataset. Basically produce the distribution of the Product column in this dataset using counters (limited to 3 counters here).

Hadoop offers Job Tracker, an UI tool to determine the status and statistics of all jobs. Using the job tracker UI, developers can view the Counters that have been created. Screenshot your  job tracker UI as your job completes and include it here. Make sure that your user defined counters are visible. 


In [89]:
!hdfs dfs -mkdir -p /hw3
!hdfs dfs -put Consumer_Complaints.csv /hw3

16/01/31 06:00:37 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/01/31 06:00:39 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
put: `/hw3/Consumer_Complaints.csv': File exists


In [1]:
%%writefile 3_1_countMapper.py
#!/usr/bin/python
#HW 3.1 - Mapper with category count

import sys

# input comes from STDIN (standard input)
for line in sys.stdin:
    line = line.strip()
    fields = line.split(',')

    product = fields[1]
    if product == "Debt collection":
        sys.stderr.write("reporter:counter:Product,Debt,1\n")
        print "Debt_collection\t1"
    elif product == "Mortgage":
        sys.stderr.write("reporter:counter:Product,Mortgage,1\n")        
        print "Mortgage\t1"
    else:
        print "Other_categories\t1"
        sys.stderr.write("reporter:counter:Product,Other,1\n")    

Overwriting 3_1_countMapper.py


In [2]:
%%writefile 3_1_reducer.py
#!/usr/bin/python
import sys

current_word = None
current_count = 0
word = None

# input comes from STDIN
for line in sys.stdin:
    line = line.strip()
    word, count = line.split('\t', 1)

    # convert count (currently a string) to int
    try:
        count = int(count)
    except ValueError:
        # count was not a number, so silently ignore/discard this line
        continue

    if current_word == word:
        current_count += count
    else:
        if current_word:
            # write result to STDOUT
            print '%s\t%s' % (current_word, current_count)
        current_count = count
        current_word = word

# output the last word
if current_word == word:
    print '%s\t%s' % (current_word, current_count)


Overwriting 3_1_reducer.py


In [3]:
!hdfs dfs -rm -r /hw3/3_1_out

!hadoop jar /usr/local/hadoop/share/hadoop/tools/lib/hadoop-streaming-2.7.1.jar \
-mapper 3_1_countMapper.py -reducer 3_1_reducer.py -input /hw3/Consumer_Complaints.csv -output /hw3/3_1_out

16/02/04 00:51:24 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/04 00:51:25 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /hw3/3_1_out
16/02/04 00:51:26 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/04 00:51:26 INFO Configuration.deprecation: session.id is deprecated. Instead, use dfs.metrics.session-id
16/02/04 00:51:26 INFO jvm.JvmMetrics: Initializing JVM Metrics with processName=JobTracker, sessionId=
16/02/04 00:51:26 INFO jvm.JvmMetrics: Cannot initialize JVM Metrics with processName=JobTracker, sessionId= - already initialized
16/02/04 00:51:26 INFO mapred.FileInputFormat: Total input paths to process : 1
16/02/04 00:51:26 INFO mapreduce.JobSubmitter: number of splits:1
16/02/04 00:51:27 INFO mapreduce.JobSubmitter: Submit

In [20]:
!hdfs dfs -cat /hw3/3_1_out/part-00000

16/01/31 03:22:58 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Debt_collection	44372
Mortgage	125752
Other_categories	142789


**Counter output screenshot**

<img src="/home/ubuntu/HW3/3_1_Counter.jpg">

### HW 3.2 Analyze the performance of your Mappers, Combiners and Reducers using Counters ###

**For this brief study the Input file will be one record (the next line only): **

foo foo quux labs foo bar quux

**Perform a word count analysis of this single record dataset using a Mapper and Reducer based WordCount (i.e., no combiners are used here) using user defined Counters to count up how many time the mapper and reducer are called. What is the value of your user defined Mapper Counter, and Reducer Counter after completing this word count job. The answer  should be 1 and 4 respectively. Please explain. **

In [21]:
%%writefile 3_2_wordcount.txt
foo foo quux labs foo bar quux

Writing 3_2_wordcount.txt


In [30]:
!hdfs dfs -put 3_2_wordcount.txt /hw3

16/01/31 04:06:24 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [50]:
%%writefile 3_2_mapper.py
#!/usr/bin/python

import sys
# increment the counter once mapper is called
sys.stderr.write("reporter:counter:Calls,mapper_calls,1\n")

for line in sys.stdin:
    # remove leading and trailing whitespace
    line = line.strip()
    words = line.split()
    for word in words:
        
        print '%s\t%s' % (word, 1)

Overwriting 3_2_mapper.py


In [28]:
%%writefile 3_2_reducer.py
#!/usr/bin/python
import sys

sys.stderr.write("reporter:counter:Calls,reducer_calls,1\n")
current_word = None
current_count = 0
word = None

# input comes from STDIN
for line in sys.stdin:
    line = line.strip()
    word, count = line.split('\t', 1)

    # convert count (currently a string) to int
    try:
        count = int(count)
    except ValueError:
        # count was not a number, so silently ignore/discard this line
        continue

    if current_word == word:
        current_count += count
    else:
        if current_word:
            # write result to STDOUT
            print '%s\t%s' % (current_word, current_count)
        current_count = count
        current_word = word

# output the last word
if current_word == word:
    print '%s\t%s' % (current_word, current_count)

Overwriting 3_2_reducer.py


In [55]:
!hdfs dfs -rm -r /hw3/3_2_out

!hadoop jar /usr/local/hadoop/share/hadoop/tools/lib/hadoop-streaming-2.7.1.jar \
-mapper 3_2_mapper.py -reducer 3_2_reducer.py -input /hw3/3_2_wordcount.txt -output /hw3/3_2_out \
-numReduceTasks 4 #Set the number of reducers to 4

16/01/31 04:23:57 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/01/31 04:23:58 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /hw3/3_2_out
16/01/31 04:23:59 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/01/31 04:24:00 INFO Configuration.deprecation: session.id is deprecated. Instead, use dfs.metrics.session-id
16/01/31 04:24:00 INFO jvm.JvmMetrics: Initializing JVM Metrics with processName=JobTracker, sessionId=
16/01/31 04:24:00 INFO jvm.JvmMetrics: Cannot initialize JVM Metrics with processName=JobTracker, sessionId= - already initialized
16/01/31 04:24:00 INFO mapred.FileInputFormat: Total input paths to process : 1
16/01/31 04:24:00 INFO mapreduce.JobSubmitter: number of splits:1
16/01/31 04:24:00 INFO mapreduce.JobSubmitter: Submit

Set the number of reducers to 4 in the command. The mapper is called once due to the small input file size. The reducer is called 4 times.

    Calls
		mapper_calls=1
		reducer_calls=4

**Please use mulitple mappers and reducers for these jobs (at least 2 mappers and 2 reducers).
Perform a word count analysis of the Issue column of the Consumer Complaints  Dataset using a Mapper and Reducer based WordCount (i.e., no combiners used anywhere)  using user defined Counters to count up how many time the mapper and reducer are called. What is the value of your user defined Mapper Counter, and Reducer Counter after completing your word count job.**

In [86]:
%%writefile 3_2_issue_mapper.py
#!/usr/bin/python
#HW 3.2

import sys
import re

WORD_RE = re.compile(r"[\w']+") #Compile regex to easily parse complete words
sys.stderr.write("reporter:counter:Calls,mapper_calls,1\n")

# input comes from STDIN (standard input)
for line in sys.stdin:
    line = line.strip()
    fields = line.split(',')
    # extract issue column
    issue=fields[3].strip()
    words=re.findall(WORD_RE, issue) #create list of words
    
    # emit the word
    for word in words:
        print '%s\t%s' % (word, 1)

Overwriting 3_2_issue_mapper.py


In [82]:
!hdfs dfs -rm -r /hw3/3_2_issue_out

!hadoop jar /usr/local/hadoop/share/hadoop/tools/lib/hadoop-streaming-2.7.1.jar \
-D mapred.map.tasks=4 \
-D mapred.reduce.tasks=4 \
-mapper 3_2_issue_mapper.py -reducer 3_2_reducer.py -input /hw3/Consumer_Complaints.csv -output /hw3/3_2_issue_out \
#-numReduceTasks 2

16/01/31 05:37:07 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/01/31 05:37:08 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /hw3/3_2_issue_out
16/01/31 05:37:09 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/01/31 05:37:10 INFO Configuration.deprecation: session.id is deprecated. Instead, use dfs.metrics.session-id
16/01/31 05:37:10 INFO jvm.JvmMetrics: Initializing JVM Metrics with processName=JobTracker, sessionId=
16/01/31 05:37:10 INFO jvm.JvmMetrics: Cannot initialize JVM Metrics with processName=JobTracker, sessionId= - already initialized
16/01/31 05:37:10 INFO mapred.FileInputFormat: Total input paths to process : 1
16/01/31 05:37:10 INFO mapreduce.JobSubmitter: number of splits:1
16/01/31 05:37:10 INFO Configuration.deprecatio

In the hadoop streaming command, set the number of mappers and reducers to 4. When the task is completed, the mapper is called once, and the reducer is called four times. The input file is split according to the block size, which overridden the suggested mapper numbers in the command.

**Perform a word count analysis of the Issue column of the Consumer Complaints  Dataset using a Mapper, Reducer, and standalone combiner (i.e., not an in-memory combiner) based WordCount using user defined Counters to count up how many time the mapper, combiner, reducer are called. What is the value of your user defined Mapper Counter, and Reducer Counter after completing your word count job.**

In [87]:
%%writefile 3_2_combiner.py
#!/usr/bin/python
import sys
import ast

# key changes and new counts
prev_key = None
total_count = 0

sys.stderr.write("reporter:counter:Calls,combiner_calls,1\n")
# Sum counts and print it
for line in sys.stdin:
    line = line.strip()
    key, count = line.split('\t', 1)
    
    try:
        count = int(count)
    except ValueError:
        # count was not a number, so silently ignore/discard this line
        continue

    if key == prev_key:
        total_count += count
    else:
        # Key changed, time to output the previous one.
        if prev_key:
            print '%s\t%s' % (prev_key, total_count)
        total_count = count
        prev_key = key

# Don't forget to output the last key that we just counted.
if prev_key == key:
    print '%s\t%s' % (key, total_count)


Writing 3_2_combiner.py


In [91]:
!hdfs dfs -rm -r /hw3/3_2_combiner_out

!hadoop jar /usr/local/hadoop/share/hadoop/tools/lib/hadoop-streaming-2.7.1.jar \
-D mapred.map.tasks=4 \
-D mapred.reduce.tasks=4 \
-mapper 3_2_issue_mapper.py -reducer 3_2_reducer.py -combiner 3_2_combiner.py -input /hw3/Consumer_Complaints.csv -output /hw3/3_2_combiner_out \
#-numReduceTasks 2

16/01/31 06:01:35 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
rm: `/hw3/3_2_combiner_out': No such file or directory
16/01/31 06:01:37 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/01/31 06:01:37 INFO Configuration.deprecation: session.id is deprecated. Instead, use dfs.metrics.session-id
16/01/31 06:01:37 INFO jvm.JvmMetrics: Initializing JVM Metrics with processName=JobTracker, sessionId=
16/01/31 06:01:37 INFO jvm.JvmMetrics: Cannot initialize JVM Metrics with processName=JobTracker, sessionId= - already initialized
16/01/31 06:01:38 INFO mapred.FileInputFormat: Total input paths to process : 1
16/01/31 06:01:38 INFO mapreduce.JobSubmitter: number of splits:1
16/01/31 06:01:38 INFO Configuration.deprecation: mapred.reduce.tasks is deprecated. Instead, use mapreduce.job.reduces
16/01/31 06:01:38 INFO Configuration

When the job is completed, the mapper is called once, the combiner is called 4 times, and the reducer is called 4 times.

**Using a single reducer: What are the top 50 most frequent terms in your word count analysis? Present the top 50 terms and their frequency and their relative frequency. If there are ties please sort the tokens in alphanumeric/string order. Present bottom 10 tokens (least frequent items). **

In [10]:
%%writefile 3_2_top_mapper.py
#!/usr/bin/python
#HW 3.2

import sys
import re

WORD_RE = re.compile(r"[\w']+") #Compile regex to easily parse complete words

# input comes from STDIN (standard input)
for line in sys.stdin:
    line = line.strip()
    fields = line.split(',')
    # extract issue column
    issue=fields[3].strip()
    words=re.findall(WORD_RE, issue) #create list of words
    
    # count the words
    for word in words:
        print '%s\t%s' % (word, 1) 
        

Overwriting 3_2_top_mapper.py


In [28]:
%%writefile 3_2_top_reducer.py
#!/usr/bin/python
import sys
word_count = {}
total_count = 0

sys.stderr.write("reporter:counter:Calls,reducer_calls,1\n")
current_key = None
current_count = 0
key = None

# input comes from STDIN
for line in sys.stdin:
    line = line.strip()
    key, count = line.split("\t", 1)
    count = int(count)
    
    if current_key == key:
        current_count += count
    else:
        if current_key:
            # write result to STDOUT
            word_count[current_key] = current_count
        current_count = count
        current_key = key

# output the last word
if current_key == key:
    word_count[current_key] = current_count

total_count = sum(word_count.values())
#word_count = sorted(word_count.items(),key=lambda t: t[1], reverse=True)

for key, value in word_count.items():
    print "%s\t%.0f\t%.4f" % (key, value, value*1.0/total_count)



Overwriting 3_2_top_reducer.py


In [29]:
!hdfs dfs -rm -r /hw3/3_2_top_out

!hadoop jar /usr/local/hadoop/share/hadoop/tools/lib/hadoop-streaming-2.7.1.jar \
-mapper 3_2_top_mapper.py -reducer 3_2_top_reducer.py -input /hw3/Consumer_Complaints.csv -output /hw3/3_2_top_out


16/02/03 01:44:43 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/03 01:44:44 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /hw3/3_2_top_out
16/02/03 01:44:45 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/03 01:44:46 INFO Configuration.deprecation: session.id is deprecated. Instead, use dfs.metrics.session-id
16/02/03 01:44:46 INFO jvm.JvmMetrics: Initializing JVM Metrics with processName=JobTracker, sessionId=
16/02/03 01:44:46 INFO jvm.JvmMetrics: Cannot initialize JVM Metrics with processName=JobTracker, sessionId= - already initialized
16/02/03 01:44:46 INFO mapred.FileInputFormat: Total input paths to process : 1
16/02/03 01:44:46 INFO mapreduce.JobSubmitter: number of splits:1
16/02/03 01:44:46 INFO mapreduce.JobSubmitter: Su

In [30]:
!hdfs dfs -cat /hw3/3_2_top_out/part-00000 |head

16/02/03 01:44:58 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Arbitration	168	0.0002
being	5663	0.0058
caused	5663	0.0058
report	34903	0.0356
attempts	11848	0.0121
Unable	4357	0.0044
issues	538	0.0005
unable	3821	0.0039
Workout	350	0.0004
credited	92	0.0001


In [31]:
!hdfs dfs -rm -r /hw3/3_2_top_out_sorted

!hadoop jar /usr/local/hadoop/share/hadoop/tools/lib/hadoop-streaming-2.7.1.jar \
-D stream.map.output.field.separator="\t" \
-D mapreduce.job.output.key.comparator.class=\
org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
 -D mapreduce.partition.keycomparator.options="-k2,2nr -k1,1" \
-mapper 3_4_mapper_sort.py -reducer 3_4_reducer_sort.py \
-input /hw3/3_2_top_out -output /hw3/3_2_top_out_sorted

16/02/03 01:45:53 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/03 01:45:53 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /hw3/3_2_top_out_sorted
16/02/03 01:45:54 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/03 01:45:55 INFO Configuration.deprecation: session.id is deprecated. Instead, use dfs.metrics.session-id
16/02/03 01:45:55 INFO jvm.JvmMetrics: Initializing JVM Metrics with processName=JobTracker, sessionId=
16/02/03 01:45:55 INFO jvm.JvmMetrics: Cannot initialize JVM Metrics with processName=JobTracker, sessionId= - already initialized
16/02/03 01:45:55 INFO mapred.FileInputFormat: Total input paths to process : 1
16/02/03 01:45:55 INFO mapreduce.JobSubmitter: number of splits:1
16/02/03 01:45:56 INFO mapreduce.JobSubmit

In [33]:
!hdfs dfs -cat /hw3/3_2_top_out_sorted/part-00000

16/02/03 01:47:03 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Loan	107254	0.1094	
	
modification	70487	0.0719	
	
credit	40483	0.0413	
	
servicing	36767	0.0375	
	
report	34903	0.0356	
	
Incorrect	29133	0.0297	
	
information	29069	0.0296	
	
on	29069	0.0296	
	
or	22533	0.0230	
	
debt	17966	0.0183	
	
Account	16555	0.0169	
	
and	16448	0.0168	
	
opening	16205	0.0165	
	
Credit	14768	0.0151	
	
club	12545	0.0128	
	
health	12545	0.0128	
	
loan	12376	0.0126	
	
not	12353	0.0126	
	
Cont'd	11848	0.0121	
	
attempts	11848	0.0121	
	
collect	11848	0.0121	
	
owed	11848	0.0121	
	
of	10885	0.0111	
	
my	10731	0.0109	
	
Deposits	10555	0.0108	
	
withdrawals	10555	0.0108	
	
Problems	9484	0.0097	
	
Application	8868	0.0090	
	
to	8401	0.0086	
	
Billing	8158	0.0083	
	
Other	7886	0.0080	
	
disputes	6938	0.0071	
	
Communication	6920	0.0071	
	
tactics	6920	0.0071	
	
reporting	6559	0.0067	
	
lease	6337	0.0065	
	
the	6248	0.0064	
	
be

### HW3.3. Shopping Cart Analysis ###

In [2]:
!hdfs dfs -put ProductPurchaseData.txt /hw3

16/02/01 22:49:14 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [34]:
%%writefile 3_3_mapper.py
#!/usr/bin/python
#HW 3.3

import sys

max_basket = 0
total_count = 0

# input comes from STDIN (standard input)
for line in sys.stdin:
    line = line.strip()
    products = line.split(" ")
    if len(products) > max_basket:
        max_basket = len(products)
    
    # count the product
    for item in products:
        print item+"\t1"
        total_count += 1

print "max_basket\t"+str(max_basket)
print "total_count\t"+str(total_count)


Overwriting 3_3_mapper.py


In [39]:
%%writefile 3_3_reducer.py
#!/usr/bin/python
import sys
from operator import itemgetter

total_count = 0
max_basket = 0
topN = 0
products = {}

sys.stderr.write("reporter:counter:Calls,reducer_calls,1\n")
current_key = None
current_count = 0
key = None

# input comes from STDIN
for line in sys.stdin:
    line = line.strip()
    key, count = line.split("\t", 1)
    count = int(count)
    
    if key == "total_count":
        total_count += count
    elif key == "max_basket":
        max_basket = max(max_basket, count)
    else:

        if current_key == key:
            current_count += count
        else:
            if current_key:
                # write result to STDOUT
                products[current_key] = current_count
            current_count = count
            current_key = key

# output the last word
if current_key == key:
    products[current_key] = current_count
#print "The number of unique items: ", len(products)
#print "The largest basket size: ", max_basket

#products = sorted(products.items(),key=lambda t: t[1], reverse=True)

#print "Product\tFrequency\tRelative frequency"    
for key, value in products.items():
    print "%s\t%.0f\t%.4f" % (key, value, value*1.0/total_count)
print "max_basket\t"+str(max_basket)
print "unique_items\t"+str(len(products))

Overwriting 3_3_reducer.py


In [40]:
!hdfs dfs -rm -r /hw3/3_3_out

!hadoop jar /usr/local/hadoop/share/hadoop/tools/lib/hadoop-streaming-2.7.1.jar \
-D stream.map.output.field.separator="\t" \
-D mapreduce.job.output.key.comparator.class=\
org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
-D mapreduce.partition.keycomparator.options="-k1,1" \
-mapper 3_3_mapper.py -reducer 3_3_reducer.py \
-input /hw3/ProductPurchaseData.txt -output /hw3/3_3_out


16/02/03 02:11:58 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/03 02:11:59 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /hw3/3_3_out
16/02/03 02:12:00 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/03 02:12:00 INFO Configuration.deprecation: session.id is deprecated. Instead, use dfs.metrics.session-id
16/02/03 02:12:00 INFO jvm.JvmMetrics: Initializing JVM Metrics with processName=JobTracker, sessionId=
16/02/03 02:12:00 INFO jvm.JvmMetrics: Cannot initialize JVM Metrics with processName=JobTracker, sessionId= - already initialized
16/02/03 02:12:00 INFO mapred.FileInputFormat: Total input paths to process : 1
16/02/03 02:12:01 INFO mapreduce.JobSubmitter: number of splits:1
16/02/03 02:12:01 INFO mapreduce.JobSubmitter: Submit

In [42]:
!hdfs dfs -cat /hw3/3_3_out/part-00000 | tail

16/02/03 02:12:44 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
FRO33948	9	0.0000
GRO18058	1	0.0000
GRO55789	1	0.0000
GRO73026	7	0.0000
FRO75698	1	0.0000
GRO43334	1	0.0000
SNA87642	1	0.0000
SNA33045	3	0.0000
max_basket	37
unique_items	12591


In [44]:
%%writefile 3_3_reducer_sort.py
#!/usr/bin/python
#HW 3.3

import sys
topN = 0
max_basket = 0
unique_items = 0

# input comes from STDIN (standard input)
for line in sys.stdin:
    line = line.strip()
    fields = line.split("\t", 1)

    if fields[0] == "max_basket":
        max_basket = int(fields[1])
    elif fields[0] == "unique_items":
        unique_items = int(fields[1])
    elif topN < 50:
        print line
        topN += 1

print "The number of unique items: ", unique_items
print "The largest basket size: ", max_basket

Overwriting 3_3_reducer_sort.py


In [45]:
!hdfs dfs -rm -r /hw3/3_3_out_sorted

!hadoop jar /usr/local/hadoop/share/hadoop/tools/lib/hadoop-streaming-2.7.1.jar \
-D stream.map.output.field.separator="\t" \
-D mapreduce.job.output.key.comparator.class=\
org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
-D mapreduce.partition.keycomparator.options="-k2,2nr -k1,1" \
-mapper 3_4_mapper_sort.py -reducer 3_3_reducer_sort.py \
-input /hw3/3_3_out -output /hw3/3_3_out_sorted

16/02/03 02:30:54 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
rm: `/hw3/3_3_out_sorted': No such file or directory
16/02/03 02:30:55 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/03 02:30:56 INFO Configuration.deprecation: session.id is deprecated. Instead, use dfs.metrics.session-id
16/02/03 02:30:56 INFO jvm.JvmMetrics: Initializing JVM Metrics with processName=JobTracker, sessionId=
16/02/03 02:30:56 INFO jvm.JvmMetrics: Cannot initialize JVM Metrics with processName=JobTracker, sessionId= - already initialized
16/02/03 02:30:56 INFO mapred.FileInputFormat: Total input paths to process : 1
16/02/03 02:30:56 INFO mapreduce.JobSubmitter: number of splits:1
16/02/03 02:30:56 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local850505182_0001
16/02/03 02:30:57 INFO mapreduce.Job: The url to track the j

In [46]:
!hdfs dfs -cat /hw3/3_3_out_sorted/part-00000

16/02/03 02:31:40 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
DAI62779	6667	0.0175
FRO40251	3881	0.0102
ELE17451	3875	0.0102
GRO73461	3602	0.0095
SNA80324	3044	0.0080
ELE32164	2851	0.0075
DAI75645	2736	0.0072
SNA45677	2455	0.0064
FRO31317	2330	0.0061
DAI85309	2293	0.0060
ELE26917	2292	0.0060
FRO80039	2233	0.0059
GRO21487	2115	0.0056
SNA99873	2083	0.0055
GRO59710	2004	0.0053
GRO71621	1920	0.0050
FRO85978	1918	0.0050
GRO30386	1840	0.0048
ELE74009	1816	0.0048
GRO56726	1784	0.0047
DAI63921	1773	0.0047
GRO46854	1756	0.0046
ELE66600	1713	0.0045
DAI83733	1712	0.0045
FRO32293	1702	0.0045
ELE66810	1697	0.0045
SNA55762	1646	0.0043
DAI22177	1627	0.0043
FRO78087	1531	0.0040
ELE99737	1516	0.0040
ELE34057	1489	0.0039
GRO94758	1489	0.0039
FRO35904	1436	0.0038
FRO53271	1420	0.0037
SNA93860	1407	0.0037
SNA90094	1390	0.0036
GRO38814	1352	0.0036
ELE56788	1345	0.0035
GRO61133	1321	0.0035
DAI88807	1316	0.0035
ELE74482	131

### HW3.4. (Computationally prohibitive but then again Hadoop can handle this) Pairs ###

In [47]:
%%writefile 3_4_mapper.py
#!/usr/bin/python
#HW 3.4

import sys
sys.stderr.write("reporter:counter:Calls,mapper_calls,1\n")
linecount = 0
# input comes from STDIN (standard input)
for line in sys.stdin:
    line = line.strip()
    products = line.split(" ")
    products = sorted(products)
    linecount += 1   
    # count the product
    for item in products:
        for item2 in products[products.index(item)+1:]:
            print "%s,%s\t1" % (item, item2)
            print "%s\t1" % (item)
    
print "linecount\t"+str(linecount)

Overwriting 3_4_mapper.py


In [49]:
%%writefile 3_4_combiner.py
#!/usr/bin/python
import sys

sys.stderr.write("reporter:counter:Calls,combiner_calls,1\n")
current_key = None
current_count = 0
key = None

# input comes from STDIN
for line in sys.stdin:
    line = line.strip()
    key, count = line.split("\t", 1)
    count = int(count)

    if current_key == key:
        current_count += count
    else:
        if current_key:
            # write result to STDOUT
            print '%s\t%s' % (current_key, current_count)
        current_count = count
        current_key = key

# output the last word
if current_key == key:
    print '%s\t%s' % (current_key, current_count)

Overwriting 3_4_combiner.py


In [204]:
%%writefile 3_4_reducer.py
#!/usr/bin/python
#HW 3.4

import sys

sys.stderr.write("reporter:counter:Calls,reducer_calls,1\n")
pairs = {}
current_key = None
current_count = 0
key = None
linecount = 0

# input comes from STDIN (standard input)
for line in sys.stdin:
    line = line.strip()
    key, count = line.split("\t", 1)
    count = int(count)
    
    if current_key == key:
        current_count += int(count)
    else:
        if current_key:
            pairs[current_key] = current_count
        current_count = count
        current_key = key

# output the last word
if current_key == key:
    pairs[current_key] = current_count
linecount = pairs["linecount"]
pairs.pop("linecount")

for key,value in pairs.items():
    if value >= 100:
        items = key.split(",", 1)
        if len(items) == 2:
            print "%s\t%s\t%s\t%.4f\t%.4f" % \
            (items[0], items[1], str(value), value*1.0/linecount, value*1.0/pairs[items[0]])

Overwriting 3_4_reducer.py


In [50]:
!hdfs dfs -rm -r /hw3/3_4_out

!hadoop jar /usr/local/hadoop/share/hadoop/tools/lib/hadoop-streaming-2.7.1.jar \
-D stream.map.output.field.separator="\t" \
-mapper 3_4_mapper.py -combiner 3_4_combiner.py -reducer 3_4_reducer.py \
-input /hw3/ProductPurchaseData.txt -output /hw3/3_4_out

16/02/03 02:36:58 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/03 02:36:59 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /hw3/3_4_out
16/02/03 02:37:00 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/03 02:37:01 INFO Configuration.deprecation: session.id is deprecated. Instead, use dfs.metrics.session-id
16/02/03 02:37:01 INFO jvm.JvmMetrics: Initializing JVM Metrics with processName=JobTracker, sessionId=
16/02/03 02:37:01 INFO jvm.JvmMetrics: Cannot initialize JVM Metrics with processName=JobTracker, sessionId= - already initialized
16/02/03 02:37:01 INFO mapred.FileInputFormat: Total input paths to process : 1
16/02/03 02:37:01 INFO mapreduce.JobSubmitter: number of splits:1
16/02/03 02:37:01 INFO mapreduce.JobSubmitter: Submit

In [51]:
!hdfs dfs -cat /hw3/3_4_out/part-00000 | head

16/02/03 02:37:39 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
DAI63921	FRO80039	126	0.0041	0.0058
DAI95741	SNA90094	120	0.0039	0.0124
DAI55911	ELE26917	113	0.0036	0.0093
DAI63921	DAI83733	142	0.0046	0.0065
DAI62779	FRO80039	550	0.0177	0.0071
DAI42493	ELE32164	266	0.0086	0.0203
ELE17451	FRO75586	151	0.0049	0.0039
SNA90094	SNA96271	104	0.0033	0.1908
DAI63921	SNA55952	110	0.0035	0.0051
ELE11111	ELE17451	121	0.0039	0.0142
cat: Unable to write to output stream.


In [282]:
%%writefile 3_4_mapper_sort.py
#!/usr/bin/python
#HW 3.4

import sys

# input comes from STDIN (standard input)
for line in sys.stdin:
    print line

Overwriting 3_4_mapper_sort.py


In [283]:
%%writefile 3_4_reducer_sort.py
#!/usr/bin/python
#HW 3.4

import sys
topN = 0

# input comes from STDIN (standard input)
for line in sys.stdin:
    if topN < 50:
        print line
        topN += 1

Overwriting 3_4_reducer_sort.py


In [52]:
!hdfs dfs -rm -r /hw3/3_4_out_sorted

!hadoop jar /usr/local/hadoop/share/hadoop/tools/lib/hadoop-streaming-2.7.1.jar \
-D stream.map.output.field.separator="\t" \
-D mapreduce.job.output.key.comparator.class=\
org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
-D mapreduce.partition.keycomparator.options="-k3,3nr -k1,1 -k2,2" \
-mapper 3_4_mapper_sort.py -reducer 3_4_reducer_sort.py \
-input /hw3/3_4_out -output /hw3/3_4_out_sorted

16/02/03 02:38:34 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/03 02:38:34 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /hw3/3_4_out_sorted
16/02/03 02:38:36 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/03 02:38:36 INFO Configuration.deprecation: session.id is deprecated. Instead, use dfs.metrics.session-id
16/02/03 02:38:36 INFO jvm.JvmMetrics: Initializing JVM Metrics with processName=JobTracker, sessionId=
16/02/03 02:38:36 INFO jvm.JvmMetrics: Cannot initialize JVM Metrics with processName=JobTracker, sessionId= - already initialized
16/02/03 02:38:36 INFO mapred.FileInputFormat: Total input paths to process : 1
16/02/03 02:38:36 INFO mapreduce.JobSubmitter: number of splits:1
16/02/03 02:38:36 INFO mapreduce.JobSubmitter:

In [53]:
!hdfs dfs -cat /hw3/3_4_out_sorted/part-00000

16/02/03 02:38:48 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
DAI62779	ELE17451	1592	0.0512	0.0204	
	
FRO40251	SNA80324	1412	0.0454	0.0489	
	
DAI75645	FRO40251	1254	0.0403	0.0365	
	
FRO40251	GRO85051	1213	0.0390	0.0420	
	
DAI62779	GRO73461	1139	0.0366	0.0146	
	
DAI75645	SNA80324	1130	0.0363	0.0329	
	
DAI62779	FRO40251	1070	0.0344	0.0137	
	
DAI62779	SNA80324	923	0.0297	0.0119	
	
DAI62779	DAI85309	918	0.0295	0.0118	
	
ELE32164	GRO59710	911	0.0293	0.0344	
	
DAI62779	DAI75645	882	0.0284	0.0113	
	
FRO40251	GRO73461	882	0.0284	0.0305	
	
DAI62779	ELE92920	877	0.0282	0.0113	
	
FRO40251	FRO92469	835	0.0268	0.0289	
	
DAI62779	ELE32164	832	0.0268	0.0107	
	
DAI75645	GRO73461	712	0.0229	0.0207	
	
DAI43223	ELE32164	711	0.0229	0.0423	
	
DAI62779	GRO30386	709	0.0228	0.0091	
	
ELE17451	FRO40251	697	0.0224	0.0180	
	
DAI85309	ELE99737	659	0.0212	0.0246	
	
DAI62779	ELE26917	650	0.0209	0.0083	
	
GRO21487	GRO73461	631	0.02

### HW3.5: Stripes ###

In [54]:
%%writefile 3_5_mapper.py
#!/usr/bin/python
#HW 3.4

import sys
sys.stderr.write("reporter:counter:Calls,mapper_calls,1\n")
linecount = 0
# input comes from STDIN (standard input)
for line in sys.stdin:
    line = line.strip()
    products = line.split(" ")
    products = sorted(products)
    linecount += 1   
    # emit the product
    for item in products:
        for item2 in products[products.index(item)+1:]:
            print "%s,%s\t1" % (item, item2)
    
print "linecount\t"+str(linecount)

Overwriting 3_5_mapper.py


In [55]:
%%writefile 3_5_reducer.py
#!/usr/bin/python
#HW 3.5

import sys

sys.stderr.write("reporter:counter:Calls,reducer_calls,1\n")
stripes = {}
current_key = None
current_count = 0
key = None
linecount = 0

# input comes from STDIN (standard input)
for line in sys.stdin:
    line = line.strip()
    key, count = line.split("\t", 1)
    count = int(count)
    
    if current_key == key:
        current_count += int(count)
    else:
        if current_key:
            items = current_key.split(",", 1)
            if len(items) == 2:
                stripes.setdefault(items[0], {})
                stripes[items[0]][items[1]]=current_count
            elif items[0] == "linecount":
                linecount = current_count
        current_count = count
        current_key = key

# output the last word
if current_key == key:
    items = current_key.split(",", 1)
    if len(items) == 2:
        stripes.setdefault(items[0], {})
        stripes[items[0]][items[1]]=current_count
    elif items[0] == "linecount":
        linecount = current_count
        

for key, stripe in stripes.items():
    marg_count = sum(stripe.values())
    for key2, count in stripe.items():
        if count >= 100:
            print "%s\t%s\t%s\t%.4f\t%.4f" % \
            (key, key2, str(count), count*1.0/linecount, count*1.0/marg_count)


Overwriting 3_5_reducer.py


In [56]:
!hdfs dfs -rm -r /hw3/3_5_out

!hadoop jar /usr/local/hadoop/share/hadoop/tools/lib/hadoop-streaming-2.7.1.jar \
-D stream.map.output.field.separator="\t" \
-mapper 3_5_mapper.py -reducer 3_5_reducer.py \
-input /hw3/ProductPurchaseData.txt -output /hw3/3_5_out

16/02/03 03:02:14 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/03 03:02:14 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /hw3/3_5_out
16/02/03 03:02:16 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/03 03:02:16 INFO Configuration.deprecation: session.id is deprecated. Instead, use dfs.metrics.session-id
16/02/03 03:02:16 INFO jvm.JvmMetrics: Initializing JVM Metrics with processName=JobTracker, sessionId=
16/02/03 03:02:16 INFO jvm.JvmMetrics: Cannot initialize JVM Metrics with processName=JobTracker, sessionId= - already initialized
16/02/03 03:02:16 INFO mapred.FileInputFormat: Total input paths to process : 1
16/02/03 03:02:16 INFO mapreduce.JobSubmitter: number of splits:1
16/02/03 03:02:17 INFO mapreduce.JobSubmitter: Submit

In [57]:
!hdfs dfs -cat /hw3/3_5_out/part-00000 | head

16/02/03 03:02:44 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
ELE20847	ELE26917	110	0.0035	0.0118
ELE20847	GRO73461	187	0.0060	0.0200
ELE20847	FRO92469	122	0.0039	0.0131
ELE20847	GRO85051	139	0.0045	0.0149
ELE20847	SNA80324	410	0.0132	0.0439
ELE20847	FRO75586	118	0.0038	0.0126
ELE20847	SNA96271	184	0.0059	0.0197
ELE20847	FRO40251	434	0.0140	0.0465
DAI22896	GRO21487	114	0.0037	0.0069
DAI22896	GRO38814	223	0.0072	0.0135


In [59]:
!hdfs dfs -rm -r /hw3/3_5_out_sorted

!hadoop jar /usr/local/hadoop/share/hadoop/tools/lib/hadoop-streaming-2.7.1.jar \
-D stream.map.output.field.separator="\t" \
-D mapreduce.job.output.key.comparator.class=\
org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
-D mapreduce.partition.keycomparator.options="-k3,3nr -k1,1 -k2,2" \
-mapper 3_4_mapper_sort.py -reducer 3_4_reducer_sort.py \
-input /hw3/3_5_out -output /hw3/3_5_out_sorted

16/02/03 03:03:27 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/03 03:03:28 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /hw3/3_5_out_sorted
16/02/03 03:03:29 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/03 03:03:30 INFO Configuration.deprecation: session.id is deprecated. Instead, use dfs.metrics.session-id
16/02/03 03:03:30 INFO jvm.JvmMetrics: Initializing JVM Metrics with processName=JobTracker, sessionId=
16/02/03 03:03:30 INFO jvm.JvmMetrics: Cannot initialize JVM Metrics with processName=JobTracker, sessionId= - already initialized
16/02/03 03:03:30 INFO mapred.FileInputFormat: Total input paths to process : 1
16/02/03 03:03:30 INFO mapreduce.JobSubmitter: number of splits:1
16/02/03 03:03:30 INFO mapreduce.JobSubmitter:

In [245]:
!hdfs dfs -cat /hw3/3_5_out_sorted/part-00000

16/02/02 05:53:01 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
DAI62779	ELE17451	1592	0.0512	0.0204	
	
FRO40251	SNA80324	1412	0.0454	0.0489	
	
DAI75645	FRO40251	1254	0.0403	0.0365	
	
FRO40251	GRO85051	1213	0.0390	0.0420	
	
DAI62779	GRO73461	1139	0.0366	0.0146	
	
DAI75645	SNA80324	1130	0.0363	0.0329	
	
DAI62779	FRO40251	1070	0.0344	0.0137	
	
DAI62779	SNA80324	923	0.0297	0.0119	
	
DAI62779	DAI85309	918	0.0295	0.0118	
	
ELE32164	GRO59710	911	0.0293	0.0344	
	
DAI62779	DAI75645	882	0.0284	0.0113	
	
FRO40251	GRO73461	882	0.0284	0.0305	
	
DAI62779	ELE92920	877	0.0282	0.0113	
	
FRO40251	FRO92469	835	0.0268	0.0289	
	
DAI62779	ELE32164	832	0.0268	0.0107	
	
DAI75645	GRO73461	712	0.0229	0.0207	
	
DAI43223	ELE32164	711	0.0229	0.0423	
	
DAI62779	GRO30386	709	0.0228	0.0091	
	
ELE17451	FRO40251	697	0.0224	0.0180	
	
DAI85309	ELE99737	659	0.0212	0.0246	
	
DAI62779	ELE26917	650	0.0209	0.0083	
	
GRO21487	GRO73461	631	0.02

In [4]:
# stop hadoop
!/usr/local/hadoop/sbin/stop-all.sh

This script is Deprecated. Instead use stop-dfs.sh and stop-yarn.sh
16/01/29 23:04:10 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Stopping namenodes on [localhost]
localhost: no namenode to stop
localhost: no datanode to stop
Stopping secondary namenodes [0.0.0.0]
0.0.0.0: no secondarynamenode to stop
16/01/29 23:04:14 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
stopping yarn daemons
no resourcemanager to stop
localhost: no nodemanager to stop
no proxyserver to stop
